# Введение

Набор данных содержит информацию о погодных условиях, зарегистрированных в каждый день на различных метеостанциях по всему миру. Информация включает осадки, снегопады, температуру, скорость ветра и то, были ли в этот день грозы или другие плохие погодные условия.

## Задание

Создать модель линейной регрессии для предсказания средней температуры (без использования минимальной и максимальной температуры напрямую), на основе предоставленных данных.

### Импорт данных и подключение библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('Summary_of_Weather.csv', low_memory=False)
data.head()

,STA,Date,Precip,WindGustSpd,MaxTemp,MinTemp,MeanTemp,Snowfall,PoorWeather,YR,...,FB,FTI,ITH,PGT,TSHDSBRSGF,SD3,RHX,RHN,RVG,WTE
0,10001,1942-7-1,1.016,NaN,25.555556,22.222222,23.888889,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,1942-7-2,0,NaN,28.888889,21.666667,25.555556,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,1942-7-3,2.54,NaN,26.111111,22.222222,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,1942-7-4,2.54,NaN,26.666667,22.222222,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,1942-7-5,0,NaN,26.666667,21.666667,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
data.PoorWeather.value_counts()

PoorWeather
1             24048
1.0            5709
0               801
1     1         284
100000          247
1 1             122
1 1  1           86
1000             86
100001           59
101000           55
1    1           53
1000001000       50
1  1             44
1000001001       42
11               37
1001             15
1   1 1          14
1100000          14
1000101001       13
101001           12
1000101000        9
11 1              7
1000000000        6
1000000           6
11    1           5
1010001000        5
1     1  1        4
1010001001        2
1 0  1            2
1   1 1  1        2
1000100001        2
1101000           2
10001001          1
1     0           1
1000100000        1
1 0               1
1101001           1
1   1             1
1        1        1
Name: count, dtype: int64

Тут представлена часть данных для наглядности содержания

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110707 entries, 0 to 110706
Data columns (total 31 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STA          110707 non-null  int64  
 1   Date         110707 non-null  object 
 2   Precip       110707 non-null  object 
 3   WindGustSpd  495 non-null     float64
 4   MaxTemp      110707 non-null  float64
 5   MinTemp      110707 non-null  float64
 6   MeanTemp     110707 non-null  float64
 7   Snowfall     109640 non-null  object 
 8   PoorWeather  31850 non-null   object 
 9   YR           110707 non-null  int64  
 10  MO           110707 non-null  int64  
 11  DA           110707 non-null  int64  
 12  PRCP         108914 non-null  object 
 13  DR           496 non-null     float64
 14  SPD          495 non-null     float64
 15  MAX          110263 non-null  float64
 16  MIN          110272 non-null  float64
 17  MEA          110243 non-null  float64
 18  SNF          109640 non-

Из анализа колонок видно, что часть из них не имеет данных, поэтому стоит исключить их из анализируемых данных.

In [4]:
data.drop(['FT', 'FB', 'FTI','ITH','SD3','RHX','RHN','RVG','WTE'], axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110707 entries, 0 to 110706
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STA          110707 non-null  int64  
 1   Date         110707 non-null  object 
 2   Precip       110707 non-null  object 
 3   WindGustSpd  495 non-null     float64
 4   MaxTemp      110707 non-null  float64
 5   MinTemp      110707 non-null  float64
 6   MeanTemp     110707 non-null  float64
 7   Snowfall     109640 non-null  object 
 8   PoorWeather  31850 non-null   object 
 9   YR           110707 non-null  int64  
 10  MO           110707 non-null  int64  
 11  DA           110707 non-null  int64  
 12  PRCP         108914 non-null  object 
 13  DR           496 non-null     float64
 14  SPD          495 non-null     float64
 15  MAX          110263 non-null  float64
 16  MIN          110272 non-null  float64
 17  MEA          110243 non-null  float64
 18  SNF          109640 non-

Также есть столбцы которые несут в себе одинаковую информацию, но в разных представлениях. Для утверждения факта идентичности столбцов сравним их. 

In [5]:
df_analyze = pd.DataFrame()
df_analyze['MaxTemp_VS_MAX'] = (data['MaxTemp'] * 1.8 + 32) - data['MAX']
df_analyze['MinTemp_VS_MIN'] = (data['MinTemp'] * 1.8 + 32) - data['MIN']
df_analyze['MeanTemp_VS_MEA'] = (data['MeanTemp'] * 1.8 + 32) - data['MEA']
df_analyze.head() 

,MaxTemp_VS_MAX,MinTemp_VS_MIN,MeanTemp_VS_MEA
0,8.000001e-09,-4.000000e-09,2.000007e-09
1,2.000007e-09,5.999993e-09,8.000001e-09
2,-2.000007e-09,-4.000000e-09,-8.000001e-09
3,5.999993e-09,-4.000000e-09,-8.000001e-09
4,5.999993e-09,5.999993e-09,-8.000001e-09


In [6]:
print('Минимальное значение по столбцам:\n', df_analyze.min())
print('Максимальное значение по столбцам:\n', df_analyze.max())

Минимальное значение по столбцам:
 MaxTemp_VS_MAX    -8.000001e-09
MinTemp_VS_MIN    -8.000001e-09
MeanTemp_VS_MEA   -8.000001e-09
dtype: float64
Максимальное значение по столбцам:
 MaxTemp_VS_MAX     8.000001e-09
MinTemp_VS_MIN     8.000001e-09
MeanTemp_VS_MEA    8.000001e-09
dtype: float64


Исходя из этого исключим из датасета значения температуры в фаренгейтах

In [7]:
data.drop(['MAX','MIN','MEA'], axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110707 entries, 0 to 110706
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STA          110707 non-null  int64  
 1   Date         110707 non-null  object 
 2   Precip       110707 non-null  object 
 3   WindGustSpd  495 non-null     float64
 4   MaxTemp      110707 non-null  float64
 5   MinTemp      110707 non-null  float64
 6   MeanTemp     110707 non-null  float64
 7   Snowfall     109640 non-null  object 
 8   PoorWeather  31850 non-null   object 
 9   YR           110707 non-null  int64  
 10  MO           110707 non-null  int64  
 11  DA           110707 non-null  int64  
 12  PRCP         108914 non-null  object 
 13  DR           496 non-null     float64
 14  SPD          495 non-null     float64
 15  SNF          109640 non-null  object 
 16  SND          5178 non-null    float64
 17  PGT          489 non-null     float64
 18  TSHDSBRSGF   31850 non-n

По описанию датасета колонки "PoorWeather" и "TSHDSBRSGF" идентичны, необходимо это проверить

In [8]:
df_analyze['PoorWeather_VS_TSHDSBRSGF'] = np.where(data['PoorWeather'].isnull(), True, data['PoorWeather'] == data['TSHDSBRSGF'])
df_analyze.PoorWeather_VS_TSHDSBRSGF.value_counts()

PoorWeather_VS_TSHDSBRSGF
True    110707
Name: count, dtype: int64

Исходя из этого можно исключить одну из колонок

In [9]:
data.drop(['TSHDSBRSGF'], axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110707 entries, 0 to 110706
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STA          110707 non-null  int64  
 1   Date         110707 non-null  object 
 2   Precip       110707 non-null  object 
 3   WindGustSpd  495 non-null     float64
 4   MaxTemp      110707 non-null  float64
 5   MinTemp      110707 non-null  float64
 6   MeanTemp     110707 non-null  float64
 7   Snowfall     109640 non-null  object 
 8   PoorWeather  31850 non-null   object 
 9   YR           110707 non-null  int64  
 10  MO           110707 non-null  int64  
 11  DA           110707 non-null  int64  
 12  PRCP         108914 non-null  object 
 13  DR           496 non-null     float64
 14  SPD          495 non-null     float64
 15  SNF          109640 non-null  object 
 16  SND          5178 non-null    float64
 17  PGT          489 non-null     float64
dtypes: float64(8), int64(4),

In [10]:
data.Precip.value_counts()

Precip
0          59746
T          15572
0.254       3145
0.508       2716
0.762       1869
           ...  
148.59         1
248.92         1
144.78         1
147.828        1
161.036        1
Name: count, Length: 531, dtype: int64

In [11]:
data.PRCP.value_counts()

PRCP
0       57953
T       15572
0.01     3145
0.02     2716
0.03     1869
        ...  
5.85        1
9.8         1
5.7         1
5.82        1
6.34        1
Name: count, Length: 531, dtype: int64

Как видно из анализа данные столбцы не получается представить в типе данных float из-за наличия кривых данных, исправим это

In [12]:
data.Snowfall = pd.to_numeric(data['Snowfall'], errors = 'coerce', downcast = 'float')
data.SNF = pd.to_numeric(data['SNF'], errors = 'coerce', downcast = 'float')
data.PRCP = pd.to_numeric(data['PRCP'], errors = 'coerce', downcast = 'float')
data.Precip = pd.to_numeric(data['Precip'], errors = 'coerce', downcast = 'float')
df_analyze['Snowfall_VS_SNF'] = data['Snowfall'] - data['SNF']  * 25.4
df_analyze['Precip_VS_PRCP'] = data['Precip'] - data['PRCP']  * 25.4
df_analyze['WindGustSpd_VS_SPD'] = data['WindGustSpd'] - data['SPD']  * 1.852
print('Минимальное значение по столбцам:\n', df_analyze.min())
print('Максимальное значение по столбцам:\n', df_analyze.max())

Минимальное значение по столбцам:
 MaxTemp_VS_MAX                   -0.0
MinTemp_VS_MIN                   -0.0
MeanTemp_VS_MEA                  -0.0
PoorWeather_VS_TSHDSBRSGF        True
Snowfall_VS_SNF             -0.000004
Precip_VS_PRCP              -0.000015
WindGustSpd_VS_SPD               -0.0
dtype: object
Максимальное значение по столбцам:
 MaxTemp_VS_MAX                    0.0
MinTemp_VS_MIN                    0.0
MeanTemp_VS_MEA                   0.0
PoorWeather_VS_TSHDSBRSGF        True
Snowfall_VS_SNF              0.000008
Precip_VS_PRCP               0.000015
WindGustSpd_VS_SPD                0.0
dtype: object


In [13]:
data.drop(['PRCP','SNF', 'SPD'], axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110707 entries, 0 to 110706
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STA          110707 non-null  int64  
 1   Date         110707 non-null  object 
 2   Precip       95135 non-null   float32
 3   WindGustSpd  495 non-null     float64
 4   MaxTemp      110707 non-null  float64
 5   MinTemp      110707 non-null  float64
 6   MeanTemp     110707 non-null  float64
 7   Snowfall     109601 non-null  float32
 8   PoorWeather  31850 non-null   object 
 9   YR           110707 non-null  int64  
 10  MO           110707 non-null  int64  
 11  DA           110707 non-null  int64  
 12  DR           496 non-null     float64
 13  SND          5178 non-null    float64
 14  PGT          489 non-null     float64
dtypes: float32(2), float64(7), int64(4), object(2)
memory usage: 11.8+ MB


Также исключим одно из представлений даты, а именно 3 столбца год, месяц и день, так как используемые библиотеки могут работать с датами в стандартном представлении

In [14]:
data.drop(['YR','MO','DA'], axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110707 entries, 0 to 110706
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STA          110707 non-null  int64  
 1   Date         110707 non-null  object 
 2   Precip       95135 non-null   float32
 3   WindGustSpd  495 non-null     float64
 4   MaxTemp      110707 non-null  float64
 5   MinTemp      110707 non-null  float64
 6   MeanTemp     110707 non-null  float64
 7   Snowfall     109601 non-null  float32
 8   PoorWeather  31850 non-null   object 
 9   DR           496 non-null     float64
 10  SND          5178 non-null    float64
 11  PGT          489 non-null     float64
dtypes: float32(2), float64(7), int64(1), object(2)
memory usage: 9.3+ MB


Также для удобства работы стоить разбить столбец с информацией о погоде "PoorWeather" на столбы по каждому событию

In [15]:
#Функция обработки строки
def myFormatStr(st):
    buf = ''
    for i in st:
        if(i == '1'):
            buf = buf + '1'
        else:
            buf = buf + '0'
    for i in range(10 - len(buf)):
        buf = buf + '0'
    return buf    
data.PoorWeather = data.apply ( lambda x: myFormatStr(str(x['PoorWeather'])), axis = 1)
data.PoorWeather.value_counts()

PoorWeather
0000000000    79658
1000000000    30104
1000001000      334
1010000000      177
1000010000      114
1010010000       98
1001000000       59
1100000000       51
1000001001       46
1000101000       23
1000101001       15
1101000000        9
1010001000        5
1100001000        5
1000100000        2
1000100001        2
1010001001        2
1000100100        1
1000000001        1
1101001000        1
Name: count, dtype: int64

Таким образом мы привели к одному представлению все данные в столбце, выполним разбиение на разные колонки

In [16]:
def getCharStrInTrueFalse(st, index):
    if st[index] == '1': 
        return True
    return False

data['Thunder'] = data.apply (lambda x: getCharStrInTrueFalse(x['PoorWeather'], 0), axis = 1)
data['Sleet'] = data.apply (lambda x: getCharStrInTrueFalse(x['PoorWeather'], 1), axis = 1)
data['Hail'] = data.apply (lambda x: getCharStrInTrueFalse(x['PoorWeather'], 2), axis = 1)
data['Dust_or_Sand'] = data.apply (lambda x: getCharStrInTrueFalse(x['PoorWeather'], 3), axis = 1)
data['Smoke_or_Haze'] = data.apply (lambda x: getCharStrInTrueFalse(x['PoorWeather'], 4), axis = 1)
data['Blowing_Snow'] = data.apply (lambda x: getCharStrInTrueFalse(x['PoorWeather'], 5), axis = 1)
data['Rain'] = data.apply (lambda x: getCharStrInTrueFalse(x['PoorWeather'], 6), axis = 1)
data['Snow'] = data.apply (lambda x: getCharStrInTrueFalse(x['PoorWeather'], 7), axis = 1)
data['Glaze'] = data.apply (lambda x: getCharStrInTrueFalse(x['PoorWeather'], 8), axis = 1)
data['Fog'] = data.apply (lambda x: getCharStrInTrueFalse(x['PoorWeather'], 9), axis = 1)
data.drop(['PoorWeather'], axis=1, inplace=True)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110707 entries, 0 to 110706
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   STA            110707 non-null  int64  
 1   Date           110707 non-null  object 
 2   Precip         95135 non-null   float32
 3   WindGustSpd    495 non-null     float64
 4   MaxTemp        110707 non-null  float64
 5   MinTemp        110707 non-null  float64
 6   MeanTemp       110707 non-null  float64
 7   Snowfall       109601 non-null  float32
 8   DR             496 non-null     float64
 9   SND            5178 non-null    float64
 10  PGT            489 non-null     float64
 11  Thunder        110707 non-null  bool   
 12  Sleet          110707 non-null  bool   
 13  Hail           110707 non-null  bool   
 14  Dust_or_Sand   110707 non-null  bool   
 15  Smoke_or_Haze  110707 non-null  bool   
 16  Blowing_Snow   110707 non-null  bool   
 17  Rain           110707 non-nul

Проанализируем полученные столбцы

In [18]:
print(data.Thunder.value_counts())
print(data.Sleet.value_counts())
print(data.Hail.value_counts())
print(data.Dust_or_Sand.value_counts())
print(data.Smoke_or_Haze.value_counts())
print(data.Blowing_Snow.value_counts())
print(data.Rain.value_counts())
print(data.Snow.value_counts())
print(data.Glaze.value_counts())
print(data.Fog.value_counts())

Thunder
False    79658
True     31049
Name: count, dtype: int64
Sleet
False    110641
True         66
Name: count, dtype: int64
Hail
False    110425
True        282
Name: count, dtype: int64
Dust_or_Sand
False    110638
True         69
Name: count, dtype: int64
Smoke_or_Haze
False    110664
True         43
Name: count, dtype: int64
Blowing_Snow
False    110495
True        212
Name: count, dtype: int64
Rain
False    110276
True        431
Name: count, dtype: int64
Snow
False    110706
True          1
Name: count, dtype: int64
Glaze
False    110707
Name: count, dtype: int64
Fog
False    110641
True         66
Name: count, dtype: int64


Добавим в датасет информацию о положении метеостанций использую еще один дата сет с описанием метеостанций

In [19]:
dataStation = pd.read_csv('Weather_Station_Locations.csv', low_memory=False)
dataStation.head()

,WBAN,NAME,STATE/COUNTRY ID,LAT,LON,ELEV,Latitude,Longitude
0,33013,AIN EL,AL,3623N,00637E,611,36.383333,6.650000
1,33031,LA SENIA,AL,3537N,00037E,88,35.616667,0.583333
2,33023,MAISON BLANCHE,AL,3643N,00314E,23,36.716667,3.216667
3,33044,TELERGMA,AL,3607N,00621E,754,36.116667,6.416667
4,12001,TINDOUF,AL,2741N,00809W,443,27.683333,-8.083333


In [20]:
dataStation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   WBAN              150 non-null    int64  
 1   NAME              150 non-null    object 
 2   STATE/COUNTRY ID  150 non-null    object 
 3   LAT               150 non-null    object 
 4   LON               150 non-null    object 
 5   ELEV              150 non-null    int64  
 6   Latitude          150 non-null    float64
 7   Longitude         150 non-null    float64
dtypes: float64(2), int64(2), object(4)
memory usage: 9.5+ KB


In [21]:
allData = pd.DataFrame()
allData = pd.merge(data, dataStation, left_on = 'STA', right_on= 'WBAN')
allData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102221 entries, 0 to 102220
Data columns (total 29 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   STA               102221 non-null  int64  
 1   Date              102221 non-null  object 
 2   Precip            88086 non-null   float32
 3   WindGustSpd       495 non-null     float64
 4   MaxTemp           102221 non-null  float64
 5   MinTemp           102221 non-null  float64
 6   MeanTemp          102221 non-null  float64
 7   Snowfall          101117 non-null  float32
 8   DR                496 non-null     float64
 9   SND               5178 non-null    float64
 10  PGT               489 non-null     float64
 11  Thunder           102221 non-null  bool   
 12  Sleet             102221 non-null  bool   
 13  Hail              102221 non-null  bool   
 14  Dust_or_Sand      102221 non-null  bool   
 15  Smoke_or_Haze     102221 non-null  bool   
 16  Blowing_Snow      10

Часть данных исключилась, так как не имеется информации о каких-то станциях

In [22]:
allData.sample(5)

,STA,Date,Precip,WindGustSpd,MaxTemp,MinTemp,MeanTemp,Snowfall,DR,SND,...,Glaze,Fog,WBAN,NAME,STATE/COUNTRY ID,LAT,LON,ELEV,Latitude,Longitude
18772,11610,1942-11-27,3.048,NaN,31.111111,22.222222,26.666667,0.0,NaN,NaN,...,False,False,11610,WALLER/BWI,TD,1037N,06113W,41,10.616667,-61.016667
26861,12101,1945-6-28,0.000,NaN,40.000000,24.444444,32.222222,0.0,NaN,NaN,...,False,False,12101,ATAR,MT,2028N,01302W,235,20.466667,-13.033333
90737,60701,1945-12-9,0.000,NaN,34.444444,25.555556,30.000000,0.0,NaN,NaN,...,False,False,60701,CANTON ISLAND,CT,0246S,17143W,3,-2.766667,-171.750000
21513,11701,1945-10-3,0.000,NaN,33.333333,21.666667,27.777778,0.0,NaN,NaN,...,False,False,11701,PORT AU PRINCE,HA,1833N,07221W,16,18.550000,-72.350000
44717,22504,1941-10-27,0.000,NaN,30.000000,22.222222,26.111111,0.0,NaN,NaN,...,False,False,22504,HICKAM/AFB,HI,2120N,15757W,3,21.333333,-157.900000


Исключим ненужные столбцы

In [23]:
allData.drop(['STA','LAT','LON','WBAN'], axis=1,inplace=True)

Таким образом мы получили датасет без повторяющихся данных и в удобном для анализа представлении

In [24]:
allData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102221 entries, 0 to 102220
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Date              102221 non-null  object 
 1   Precip            88086 non-null   float32
 2   WindGustSpd       495 non-null     float64
 3   MaxTemp           102221 non-null  float64
 4   MinTemp           102221 non-null  float64
 5   MeanTemp          102221 non-null  float64
 6   Snowfall          101117 non-null  float32
 7   DR                496 non-null     float64
 8   SND               5178 non-null    float64
 9   PGT               489 non-null     float64
 10  Thunder           102221 non-null  bool   
 11  Sleet             102221 non-null  bool   
 12  Hail              102221 non-null  bool   
 13  Dust_or_Sand      102221 non-null  bool   
 14  Smoke_or_Haze     102221 non-null  bool   
 15  Blowing_Snow      102221 non-null  bool   
 16  Rain              10